In [2]:
import requests

# Using a Cerner Test Server

In [35]:
CERNER_BASE_URL = "https://fhir-myrecord.stagingcerner.com/beta/ec2458f2-1e24-41c8-b71b-0e701af7583d/"
NANCY_PATIENT_ID = "12724066"

## Scenario 0

In [4]:
DISCOVERY_PATH = ".well-known/smart-configuration"

In [5]:
headers = {"Accept" : "application/fhir+json"}
cerner_config = requests.get(CERNER_BASE_URL + DISCOVERY_PATH, headers=headers).json()

Payload includes `permission_v2` and `authorize-post` capabilities ✔️:

In [6]:
cerner_config["capabilities"]

['launch-ehr',
 'launch-standalone',
 'client-public',
 'client-confidential-symmetric',
 'sso-openid-connect',
 'context-banner',
 'context-style',
 'context-standalone-patient',
 'context-ehr-patient',
 'permission-patient',
 'permission-user',
 'permission-offline',
 'permission-v2',
 'authorize-post']

`code_challenge_methods_supported` includes `S256` ✔️:

In [7]:
cerner_config["code_challenge_methods_supported"]

['S256']

`introscpection_endpoint` included ✔️:

In [8]:
cerner_config["introspection_endpoint"]

'https://authorization.cerner.com/tokeninfo'

## Scenario 1

In [32]:
authorize_url = cerner_config["authorization_endpoint"]
token_url = cerner_config["token_endpoint"]

redirect_uri = "http://localhost:8080/index.html"

f = open("CERNER_CLIENT_ID.txt", "r")
client_id = f.read()
f.close()

aud = "https%3A%2F%2Ffhir-myrecord.cerner.com%2Fr4%2Fec2458f2-1e24-41c8-b71b-0e701af7583d"

scopes = "patient/Observation.read%20patient/Observation.rs%20patient/Observation.crs"

authorization_redirect_url = "{}?client_id={}&response_type=code&scope={}&redirect_uri={}&aud={}".format(authorize_url, 
                                                                                                  client_id, 
                                                                                                  scopes, 
                                                                                                  redirect_uri,
                                                                                                  aud)
    
print("Navigate to: ", authorization_redirect_url + "\n")

authorization_code = input('authz code: ')

headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'grant_type': 'authorization_code', 'code': authorization_code, 'redirect_uri': redirect_uri, 'client_id': client_id}
access_token_response = requests.post(token_url, data=data, headers=headers)

print("\nScopes Granted: " + access_token_response.json()['scope'])

Navigate to:  https://authorization.cerner.com/tenants/ec2458f2-1e24-41c8-b71b-0e701af7583d/protocols/oauth2/profiles/smart-v1/personas/patient/authorize?client_id=09a2a7f5-68b6-451a-b5dc-547889cf63b4&response_type=code&scope=patient/Observation.read%20patient/Observation.rs%20patient/Observation.crs&redirect_uri=http://localhost:8080/index.html&aud=https%3A%2F%2Ffhir-myrecord.cerner.com%2Fr4%2Fec2458f2-1e24-41c8-b71b-0e701af7583d

authz code: ca1f126c-2d25-4229-89d2-16a556d1559d

Scopes Granted: patient/Observation.read patient/Observation.rs patient/Observation.crs


Request `Observation`'s for a specific patient ✔️:

In [36]:
headers = {"Accept" : "application/fhir+json", "Authorization" : "Bearer " + access_token_response.json()['access_token']}
data_request_response = requests.get("{}Observation?patient={}".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response.text)

{"resourceType":"Bundle","id":"f24d921d-b666-4ac1-b632-93a238ed2a55","type":"searchset","link":[{"relation":"self","url":"https://fhir-myrecord.stagingcerner.com/beta/ec2458f2-1e24-41c8-b71b-0e701af7583d/Observation?patient=12724066"},{"relation":"next","url":"https://fhir-myrecord.stagingcerner.com/beta/ec2458f2-1e24-41c8-b71b-0e701af7583d/Observation?patient=12724066\u0026-pageContext=T3BlblBsYXRmb3JtRmhpckNvbnRleHQ9dHJ1ZSZwYWdlQ29udGV4dD05ZWQ4Y2I4MC1hYTc4LTQ0Y2QtYTEyNC0yYTg0ZDRhZWMyMDImY29uY2VwdD1jaGFydGVkX29ic2VydmF0aW9u\u0026-pageDirection=NEXT"}],"entry":[{"fullUrl":"https://fhir-myrecord.stagingcerner.com/beta/ec2458f2-1e24-41c8-b71b-0e701af7583d/Observation/SH-1-73080191","resource":{"resourceType":"Observation","id":"SH-1-73080191","meta":{"versionId":"73080191","lastUpdated":"2020-06-11T04:05:33.000Z"},"text":{"status":"generated","div":"\u003Cdiv xmlns=\"http://www.w3.org/1999/xhtml\"\u003E\u003Cp\u003E\u003Cb\u003EObservation\u003C/b\u003E\u003C/p\u003E\u003Cp\u003E\u003Cb\

Request for all `Observation`'s should fail with `400` ✔️:

In [38]:
data_request_response = requests.get("{}Observation".format(CERNER_BASE_URL), headers=headers)

print(data_request_response)

<Response [400]>


Request for insufficient scope should fail with `403` ✔️:

In [39]:
data_request_response = requests.get("{}Encounter?patient={}".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [403]>


## Scenario 2

Request and be granted the `patient/Observation.rs?category=http://terminology.hl7.org/CodeSystem/observation-category|vital-signs` and `patient/Observation.crs?category=http://terminology.hl7.org/CodeSystem/observation-category|vital-signs` scopes ✔️: 

In [44]:
authorize_url = cerner_config["authorization_endpoint"]
token_url = cerner_config["token_endpoint"]

redirect_uri = "http://localhost:8080/index.html"

f = open("CERNER_CLIENT_ID.txt", "r")
client_id = f.read()
f.close()

aud = "https%3A%2F%2Ffhir-myrecord.cerner.com%2Fr4%2Fec2458f2-1e24-41c8-b71b-0e701af7583d"

scopes = "patient/Observation.read%20patient%2FObservation.rs%3Fcategory%3Dhttp%3A%2F%2Fterminology.hl7.org%2FCodeSystem%2Fobservation-category%7Cvital-signs%20patient%2FObservation.crs%3Fcategory%3Dhttp%3A%2F%2Fterminology.hl7.org%2FCodeSystem%2Fobservation-category%7Cvital-signs"

authorization_redirect_url = "{}?client_id={}&response_type=code&scope={}&redirect_uri={}&aud={}".format(authorize_url, 
                                                                                                  client_id, 
                                                                                                  scopes, 
                                                                                                  redirect_uri,
                                                                                                  aud)
    
print("Navigate to: ", authorization_redirect_url + "\n")

authorization_code = input('authz code: ')

headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'grant_type': 'authorization_code', 'code': authorization_code, 'redirect_uri': redirect_uri, 'client_id': client_id}
access_token_response = requests.post(token_url, data=data, headers=headers)

print("\nScopes Granted: " + access_token_response.json()['scope'])

Navigate to:  https://authorization.cerner.com/tenants/ec2458f2-1e24-41c8-b71b-0e701af7583d/protocols/oauth2/profiles/smart-v1/personas/patient/authorize?client_id=09a2a7f5-68b6-451a-b5dc-547889cf63b4&response_type=code&scope=patient/Observation.read%20patient%2FObservation.rs%3Fcategory%3Dhttp%3A%2F%2Fterminology.hl7.org%2FCodeSystem%2Fobservation-category%7Cvital-signs%20patient%2FObservation.crs%3Fcategory%3Dhttp%3A%2F%2Fterminology.hl7.org%2FCodeSystem%2Fobservation-category%7Cvital-signs&redirect_uri=http://localhost:8080/index.html&aud=https%3A%2F%2Ffhir-myrecord.cerner.com%2Fr4%2Fec2458f2-1e24-41c8-b71b-0e701af7583d

authz code: 0911a8a4-2f37-42fb-b8ab-5b777db44916

Scopes Granted: patient/Observation.read patient/Observation.rs?category=http://terminology.hl7.org/CodeSystem/observation-category|vital-signs patient/Observation.crs?category=http://terminology.hl7.org/CodeSystem/observation-category|vital-signs


Expect to recieve `Observation` `vital-signs` for Nancy ✔️:

In [46]:
headers = {"Accept" : "application/fhir+json", "Authorization" : "Bearer " + access_token_response.json()['access_token']}
data_request_response = requests.get("{}Observation?patient={}&category=vital-signs".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [200]>


Expect to (again) recieve `Observation` `vital-signs` for Nancy (different way of formatting the query here) ✔️:

In [47]:
data_request_response = requests.get("{}Observation?patient={}&category=http://terminology.hl7.org/CodeSystem/observation-category|vital-signs".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [200]>


Expect to fail when `category` is not set ✔️:

In [48]:
data_request_response = requests.get("{}Observation?patient={}".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [422]>


Expect to fail when out of scope `category` is set ✔️:

In [49]:
data_request_response = requests.get("{}Observation?patient={}&category=labratory".format(CERNER_BASE_URL, NANCY_PATIENT_ID), headers=headers)

print(data_request_response)

<Response [422]>
